### Shapes in CNNs
Shapes in the Feature Extractors

In [11]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

N, n_H, n_W, n_c = 32, 28, 28, 3
n_conv_filter = 5
k_size = 3
pool_size, pool_strides = 2, 2
batch_size = 32

x = tf.random.normal(shape=(N, n_H, n_W, n_c))

conv1 = Conv2D(filters=n_conv_filter, kernel_size=k_size, padding='same', activation='relu')
conv1_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

conv2 = Conv2D(filters=n_conv_filter, kernel_size=k_size, padding='same', activation='relu')
conv2_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

flatten = Flatten()

print("input: ", x.shape)
print("----")

x = conv1(x)
W, B = conv1.get_weights()
print("W: ", W.shape, ", B: ", B.shape)
print("after conv1: ", x.shape)
print("----")

x = conv1_pool(x)
print("after conv1 pool: ", x.shape)
print("----")

x = conv2(x)
W, B = conv2.get_weights()
print("W: ", W.shape, ", B: ", B.shape)
print("after conv2: ", x.shape)
print("----")

x = conv2_pool(x)
print("after conv2 pool: ", x.shape)
print("----")

x = flatten(x)
print("after flatten: ", x.shape)


input:  (32, 28, 28, 3)
----
W:  (3, 3, 3, 5) , B:  (5,)
after conv1:  (32, 28, 28, 5)
----
after conv1 pool:  (32, 14, 14, 5)
----
W:  (3, 3, 5, 5) , B:  (5,)
after conv2:  (32, 14, 14, 5)
----
after conv2 pool:  (32, 7, 7, 5)
----
after flatten:  (32, 245)


Shapes in the Classifier

In [12]:
from tensorflow.keras.layers import Dense

n_neurons = [50, 25, 10]

dense1 = Dense(units=n_neurons[0], activation='relu')
dense2 = Dense(units=n_neurons[1], activation='relu')
dense3 = Dense(units=n_neurons[2], activation='softmax')

print("Input features: ", x.shape)
print("----")

x = dense1(x)
W, B = dense1.get_weights()
print("W: ", W.shape, ", B: ", B.shape)
print("after dense1: ", x.shape)
print("----")

x = dense2(x)
W, B = dense2.get_weights()
print("W: ", W.shape, ", B: ", B.shape)
print("after dense2: ", x.shape)
print("----")

x = dense3(x)
W, B = dense3.get_weights()
print("W: ", W.shape, ", B: ", B.shape)
print("after dense3: ", x.shape)


Input features:  (32, 245)
----
W:  (245, 50) , B:  (50,)
after dense1:  (32, 50)
----
W:  (50, 25) , B:  (25,)
after dense2:  (32, 25)
----
W:  (25, 10) , B:  (10,)
after dense3:  (32, 10)


Shapes in the Loss Functions

In [15]:
from tensorflow.keras.losses import CategoricalCrossentropy

y = tf.random.uniform(minval=0, maxval=10, shape=(32, ), dtype=tf.int32)
y = tf.one_hot(y, depth=10)

loss_object = CategoricalCrossentropy()
loss = loss_object(y, x)
print(loss.shape)

()


### Implementation of CNNs
Implementation with Sequential Method

In [16]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


N, n_H, n_W, n_c = 32, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_c))

model = Sequential()

model.add(Conv2D(filters=n_conv_neurons[0], kernel_size=k_size, padding=padding, activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Conv2D(filters=n_conv_neurons[1], kernel_size=k_size, padding=padding, activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Conv2D(filters=n_conv_neurons[2], kernel_size=k_size, padding=padding, activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Flatten())

model.add(Dense(units=n_neurons[0], activation=activation))
model.add(Dense(units=n_neurons[1], activation=activation))
model.add(Dense(units=n_neurons[2], activation='softmax'))

predictions = model(x)
print(predictions.shape)

(32, 10)


In [17]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


N, n_H, n_W, n_c = 32, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_c))

model = Sequential()

for n_conv_neuron in n_conv_neurons:
    model.add(Conv2D(filters=n_conv_neuron, kernel_size=k_size, padding=padding, activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Flatten())

for n_dense_neuron in n_dense_neurons:
    model.add(Dense(units=n_dense_neuron, activation=activation))

model.add(Dense(units=n_neurons[-1], activation='softmax'))

predictions = model(x)
print(predictions.shape)

(32, 10)


Implementation with Model Sub-classing

In [20]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()

        self.conv1 = Conv2D(filters=n_conv_neurons[0], kernel_size=k_size, padding=padding, activation=activation)
        self.conv1_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

        self.conv2 = Conv2D(filters=n_conv_neurons[1], kernel_size=k_size, padding=padding, activation=activation)
        self.conv2_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

        self.conv3 = Conv2D(filters=n_conv_neurons[2], kernel_size=k_size, padding=padding, activation=activation)
        self.conv3_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

        self.flatten = Flatten()

        self.dense1 = Dense(units=n_neurons[0], activation=activation)
        self.dense2 = Dense(units=n_neurons[1], activation=activation)
        self.dense3 = Dense(units=n_neurons[2], activation='softmax')

    def call(self, x):
        print(x.shape)

        x = self.conv1(x)
        print(x.shape)
        x = self.conv1_pool(x)
        print(x.shape)

        x = self.conv2(x)
        print(x.shape)
        x = self.conv2_pool(x)
        print(x.shape)

        x = self.conv3(x)
        print(x.shape)
        x = self.conv3_pool(x)
        print(x.shape)

        x = self.flatten(x)
        print(x.shape)

        x = self.dense1(x)
        print(x.shape)
        x = self.dense2(x)
        print(x.shape)
        x = self.dense3(x)
        print(x.shape)
        
        return x


N, n_H, n_W, n_c = 32, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2
activation = 'relu'

x = tf.random.normal(shape=(N, n_H, n_W, n_c))

model = TestCNN()
y = model(x)

(32, 28, 28, 3)
(32, 28, 28, 10)
(32, 14, 14, 10)
(32, 14, 14, 20)
(32, 7, 7, 20)
(32, 7, 7, 30)
(32, 3, 3, 30)
(32, 270)
(32, 50)
(32, 25)
(32, 10)


Implementation with Sequential + Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]

class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x
    

model = Sequential()
model.add(MyConv(n_conv_neurons[0]))
model.add(MyConv(n_conv_neurons[1]))
model.add(MyConv(n_conv_neurons[2]))

model.add(Flatten())

model.add(Dense(units=n_dense_neurons[0], activation=activation))
model.add(Dense(units=n_dense_neurons[1], activation=activation))
model.add(Dense(units=n_dense_neurons[2], activation='softmax'))

Implementation with Model and Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]


class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x


class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()

        self.conv1 = MyConv(n_conv_neurons[0])
        self.conv2 = MyConv(n_conv_neurons[1])
        self.conv3 = MyConv(n_conv_neurons[2])
        self.flatten = Flatten()

        self.dense1 = Dense(units=n_dense_neurons[0], activation=activation)
        self.dense2 = Dense(units=n_dense_neurons[1], activation=activation)
        self.dense3 = Dense(units=n_dense_neurons[2], activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)

        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]


class MyConv(Layer):
    def __init__(self, n_neuron):
        super(MyConv, self).__init__()

        self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, padding=padding, activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x


class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()
        
        self.fe = Sequential()
        self.fe.add(MyConv(n_conv_neurons[0]))
        self.fe.add(MyConv(n_conv_neurons[1]))
        self.fe.add(MyConv(n_conv_neurons[2]))
        self.fe.add(Flatten())

        self.classifier = Sequential()
        self.classifier.add(Dense(units=n_dense_neurons[0], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[1], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[2], activation='softmax'))

    def call(self, x):
        x = self.fe(x)
        x = self.classifier(x)
        return x